In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline
# plt.show()
import warnings

warnings.filterwarnings('ignore')

from pandas_datareader import data as pdr
import yfinance as yf
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import requests
import datetime
#from datetime import datetime
import time

In [2]:
open_df_original = pd.read_csv('open_2020-03-10_2021-03-10_4635', index_col='date')
high_df_original = pd.read_csv('high_2020-03-10_2021-03-10_4635', index_col='date')
low_df_original = pd.read_csv('low_2020-03-10_2021-03-10_4635', index_col='date')
adjclose_df_original = pd.read_csv('adjclose_2020-03-10_2021-03-10_4635', index_col='date')
volume_df_original = pd.read_csv('volume_2020-03-10_2021-03-10_4635', index_col='date')

In [3]:
period_filter_start_date = '2020-03-10'
period_filter_end_date = '2021-03-13'
open_df = open_df_original.loc[period_filter_start_date:period_filter_end_date]
high_df = high_df_original.loc[period_filter_start_date:period_filter_end_date]
low_df = low_df_original.loc[period_filter_start_date:period_filter_end_date]
adjclose_df = adjclose_df_original.loc[period_filter_start_date:period_filter_end_date]
volume_df = volume_df_original.loc[period_filter_start_date:period_filter_end_date]

In [4]:
volumes = pd.DataFrame(volume_df.mean(), columns= ['vol avg'])

# get shares having volume lower than 25th percentile
quantile_value=.25
vol_filter = pd.DataFrame(list(zip(volumes.index, volumes['vol avg'])), columns=['Ticker', 'Vol Avg'])
vol_filter['Rank'] =  vol_filter['Vol Avg'].rank(pct=True) * 100
df_filtered = vol_filter[vol_filter['Rank'] <= vol_filter['Rank'].quantile(quantile_value)]
df_filtered

,Ticker,Vol Avg,Rank
5,AAMC,9098.812,3.883495
24,ABR-PA,5638.062,2.761597
25,ABR-PB,4438.880,2.243797
26,ABR-PC,5647.598,2.783172
29,ABTX,76805.752,17.626753
...,...,...,...
4592,YMTX,92568.816,19.288026
4594,YORW,40544.028,12.168285
4610,ZEAL,7476.740,3.451996
4612,ZEUS,53303.016,14.714132


# DA AGGIUNGERE UN GROUP BY PER SETTORE E FARE IL COUNT PER SECTOR

### Filter in other dataframes the actual stocks subset and drop NaN  in dataframes (probably the stock which are dropped down are not tradable anymore)

In [5]:
# Filter in other dataframes the actual stocks subset and drop NaN 
# in dataframes (probably the stock which are dropped down are not tradable anymore)
open_df = open_df[list(df_filtered['Ticker'].values)]
open_df.dropna(axis = 'columns',inplace = True)
high_df = high_df[list(df_filtered['Ticker'].values)]
high_df.dropna(axis = 'columns',inplace = True)
low_df = low_df[list(df_filtered['Ticker'].values)]
low_df.dropna(axis = 'columns',inplace = True)
adjclose_df = adjclose_df[list(df_filtered['Ticker'].values)]
adjclose_df.dropna(axis = 'columns',inplace = True)
volume_df = volume_df[list(df_filtered['Ticker'].values)]
volume_df.dropna(axis = 'columns',inplace = True)

### Adjusted close price correlation matrix

In [6]:
# correlation matrix of our stocks subset
corr = adjclose_df.corr()
corr

,AAMC,ABR-PA,ABR-PB,ABR-PC,ABTX,AC,ACBI,ACGLP,ACH,ACNB,...,XGN,XOMA,XTLB,XYF,Y,YMTX,YORW,ZEAL,ZEUS,ZNH
AAMC,1.000000,0.721702,0.756409,0.711911,0.703966,0.106115,0.710405,0.744680,0.589060,0.230071,...,0.308280,0.649037,0.774749,-0.067713,0.668237,-0.616082,0.288883,0.327739,0.739709,0.860809
ABR-PA,0.721702,1.000000,0.986354,0.983573,0.518803,0.487215,0.493379,0.897054,0.498254,0.027599,...,0.068893,0.458081,0.649906,-0.175041,0.450511,-0.284592,0.565242,0.519843,0.602444,0.651109
ABR-PB,0.756409,0.986354,1.000000,0.981721,0.535452,0.448528,0.504333,0.906094,0.506654,0.019534,...,0.086872,0.475853,0.670456,-0.207540,0.486656,-0.323407,0.535011,0.528731,0.620375,0.683523
ABR-PC,0.711911,0.983573,0.981721,1.000000,0.502689,0.473014,0.468222,0.874927,0.474707,0.007881,...,0.050190,0.453615,0.634929,-0.184175,0.414359,-0.243170,0.538341,0.525939,0.584549,0.632749
ABTX,0.703966,0.518803,0.535452,0.502689,1.000000,0.006491,0.951814,0.527800,0.869614,0.690306,...,0.558819,0.903644,0.893946,0.231576,0.858186,-0.672272,0.311540,0.013544,0.920560,0.781877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YMTX,-0.616082,-0.284592,-0.323407,-0.243170,-0.672272,0.266820,-0.724349,-0.395347,-0.604796,-0.313657,...,-0.365953,-0.596769,-0.680856,0.014937,-0.682555,1.000000,-0.042619,0.106082,-0.682658,-0.682184
YORW,0.288883,0.565242,0.535011,0.538341,0.311540,0.486521,0.182515,0.593471,0.282900,0.153473,...,-0.174567,0.297072,0.341218,-0.145937,0.319165,-0.042619,1.000000,0.479533,0.328677,0.259846
ZEAL,0.327739,0.519843,0.528731,0.525939,0.013544,0.361703,-0.063083,0.533597,-0.060353,-0.216787,...,-0.282446,0.030625,0.163299,-0.418309,0.048747,0.106082,0.479533,1.000000,0.122460,0.157136
ZEUS,0.739709,0.602444,0.620375,0.584549,0.920560,0.055969,0.891216,0.604023,0.896456,0.576911,...,0.464775,0.777548,0.872216,0.164271,0.794542,-0.682658,0.328677,0.122460,1.000000,0.860334


### Correlation pairs (Strong correlation level --> .9)

In [7]:
corr_threshold = .9 # editable
corr_pairs = corr.unstack()
strong_pairs = corr_pairs[(abs(corr_pairs) > corr_threshold) & (corr_pairs != 1)]
strong_pairs

ABR-PA  ABR-PB    0.986354
        ABR-PC    0.983573
        AIC       0.946142
        AIW       0.941964
        ANH-PA    0.952079
                    ...   
ZNH     BRK-A     0.910942
        CAI       0.904275
        CEA       0.955537
        CIGI      0.902201
        LAWS      0.900891
Length: 68756, dtype: float64

### Get the list of strong pair stocks we filtered from the entire universe 

In [8]:
stocks = []
for item in strong_pairs.index:
    stocks.append(item[0])
stocks = list(dict.fromkeys(stocks))
stocks

['ABR-PA',
 'ABR-PB',
 'ABR-PC',
 'ABTX',
 'ACBI',
 'ACGLP',
 'ACH',
 'ACU',
 'ADUS',
 'AEB',
 'AEYE',
 'AGM',
 'AGX',
 'AHL-PC',
 'AHT-PD',
 'AIC',
 'AIRG',
 'AIW',
 'AJX',
 'AKO-A',
 'AKO-B',
 'ALG',
 'ALIN-PA',
 'ALIN-PB',
 'ALIN-PE',
 'ALOT',
 'ALTA',
 'ALTG',
 'AMAL',
 'AMCIU',
 'AMEH',
 'AMH-PD',
 'AMNB',
 'AMOT',
 'AMRB',
 'AMRK',
 'AMWD',
 'ANAT',
 'ANH-PA',
 'ANH-PB',
 'ANH-PC',
 'AP',
 'APO-PA',
 'APYX',
 'ARD',
 'AROW',
 'ASB-PC',
 'ASB-PD',
 'ASR',
 'ASRVP',
 'ASTE',
 'ASUR',
 'ASYS',
 'ATCO-PD',
 'ATCO-PG',
 'ATCO-PH',
 'ATLC',
 'ATLO',
 'ATTO',
 'AVD',
 'AVNW',
 'AWRE',
 'AWX',
 'AXR',
 'AZRE',
 'BAC-PE',
 'BAC-PL',
 'BANF',
 'BASI',
 'BATRA',
 'BBSI',
 'BBU',
 'BCBP',
 'BCH',
 'BCML',
 'BCPC',
 'BCYC',
 'BDL',
 'BELFA',
 'BELFB',
 'BF-A',
 'BFIN',
 'BFST',
 'BGSF',
 'BH',
 'BHB',
 'BIO-B',
 'BML-PG',
 'BML-PH',
 'BML-PJ',
 'BML-PL',
 'BMRC',
 'BMTC',
 'BNSO',
 'BOCH',
 'BOMN',
 'BOTJ',
 'BPOPM',
 'BPOPN',
 'BPYUP',
 'BRK-A',
 'BRPA',
 'BRT',
 'BSET',
 'BSRR',
 'BSVN',
 '

### Extract single pairs

In [9]:
%%time
# extract single pairs
a = []
s = []
for i, k in strong_pairs.index:
    if ([i, k] in a) == False:  ### ????? perché non consideri anche [k, i] in a ????????
        s.append([i, k])
    a.append([i,k])
    a.append([k,i])

### Select just the strong pair stocks in our imported data for adjusted close price (you can also try with other prices, ex. close or open)

In [10]:
c_data = adjclose_df[stocks]

### Train&Test 

In [11]:
# split train and test
train_size = .725
train_end = round(len(c_data.index) * train_size)
c_data_train = adjclose_df.iloc[0:train_end]
c_data_test = adjclose_df.iloc[train_end:]

### Cointegration Test

In [12]:
# run cointegratioin test on the pairs
import statsmodels.formula.api as sm
import statsmodels.tsa.stattools as ts
import statsmodels.tsa.vector_ar.vecm as vm

In [65]:
res = {}
halflife_lower_threshold = 0
halflife_upper_threshold = 30

for i, k in s:
    result=vm.coint_johansen(c_data_train[[i,  k]].values, det_order=0, k_ar_diff=1)
    if (result.trace_stat[0] > result.trace_stat_crit_vals[0][1]) & (True in (result.trace_stat[1] > result.trace_stat_crit_vals[1])):
        if (result.max_eig_stat[0] > result.max_eig_stat_crit_vals[0][1]):
            
            yport=pd.DataFrame(np.dot(c_data_train[[i,  k]].values, result.evec[:, 0])) #  (net) market value of portfolio

            # compute half life
            ylag=yport.shift()
            deltaY=yport-ylag
            df2=pd.concat([ylag, deltaY], axis=1)
            df2.columns=['ylag', 'deltaY']
            regress_results=sm.ols(formula="deltaY ~ ylag", data=df2).fit()

            halflife=-np.log(2)/regress_results.params['ylag']
            
            if halflife_lower_threshold < halflife < halflife_upper_threshold:
                lookback=np.round(halflife).astype(int) #  setting lookback to the halflife found above
                numUnits =-(yport-yport.rolling(lookback).mean())/yport.rolling(lookback).std() 
                
                positions=pd.DataFrame(np.dot(numUnits.values, np.expand_dims(result.evec[:, 0], axis=1).T)*c_data_test[[i,  k]].values)
                pnl=np.sum((positions.shift().values)*(c_data_test[[i,  k]].pct_change().values), axis=1) # daily P&L of the strategy
                ret=pnl/np.sum(np.abs(positions.shift()), axis=1)
                cumret = pd.DataFrame((np.cumprod(1+ret)-1))

                res[i+'/'+k] = {'position': positions, 'yport':yport, 'halflike': halflife, 'pnl': pnl, 'ret': ret,'cumret': cumret}



ValueError: operands could not be broadcast together with shapes (181,2) (69,2) 

In [14]:
res = {}
i = s[0][0]
k = s[0][1]
halflife_lower_threshold = 0
halflife_upper_threshold = 30

result=vm.coint_johansen(c_data_train[[i,  k]].values, det_order=0, k_ar_diff=1)
if (result.trace_stat[0] > result.trace_stat_crit_vals[0][1]) & (True in (result.trace_stat[1] > result.trace_stat_crit_vals[1])):
        if (result.max_eig_stat[0] > result.max_eig_stat_crit_vals[0][1]):
            yport=pd.DataFrame(np.dot(c_data_train[[i,  k]].values, result.evec[:, 0])) #  (net) market value of portfolio
            # compute half life
            ylag=yport.shift()
            deltaY=yport-ylag
            df2=pd.concat([ylag, deltaY], axis=1)
            df2.columns=['ylag', 'deltaY']
            regress_results=sm.ols(formula="deltaY ~ ylag", data=df2).fit()

            halflife=-np.log(2)/regress_results.params['ylag']
            if halflife_lower_threshold < halflife < halflife_upper_threshold:
                lookback=np.round(halflife).astype(int) #  setting lookback to the halflife found above
                numUnits =-(yport-yport.rolling(lookback).mean())/yport.rolling(lookback).std() 
                positions=pd.DataFrame(np.dot(numUnits.values, np.expand_dims(result.evec[:, 0], axis=1).T)*c_data_test[[i,  k]].values)


ValueError: operands could not be broadcast together with shapes (181,2) (69,2) 

In [28]:
c_data_test

,AAMC,ABR-PA,ABR-PB,ABR-PC,ABTX,AC,ACBI,ACGLP,ACH,ACNB,...,XGN,XOMA,XTLB,XYF,Y,YMTX,YORW,ZEAL,ZEUS,ZNH
date,,,,,,,,,,,,,,,,,,,,,
2020-11-24,22.650000,25.510000,25.389999,25.780001,33.118752,38.320286,16.379999,25.512508,9.24,26.626890,...,15.250000,32.500000,2.40,2.20,609.169983,20.200001,46.843277,35.830002,15.779731,32.369999
2020-11-25,22.600000,25.500099,25.389999,25.879999,32.710003,37.432594,15.590000,25.394028,8.99,26.924067,...,14.630000,32.660000,2.49,2.36,597.719971,21.000000,46.345261,36.301998,15.240423,31.799999
2020-11-27,22.760000,25.570000,25.389999,25.848301,32.400002,38.430000,15.210000,25.324915,9.27,26.299999,...,14.750000,32.049999,2.51,2.75,589.880005,21.799999,45.847248,37.629002,15.570001,31.500000
2020-11-30,23.049999,25.455000,25.250000,25.790001,31.660000,38.250000,14.040000,25.413774,9.82,24.910000,...,14.860000,33.110001,2.46,2.50,575.200012,21.200001,44.861179,38.529999,14.840000,30.930000
2020-12-01,23.000000,25.473700,25.209999,25.950001,32.610001,37.610001,14.430000,25.492762,9.83,25.500000,...,14.260000,32.759998,2.42,2.45,590.750000,20.400000,45.857208,37.340000,15.030000,31.840000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-01,23.639999,26.080000,25.416401,25.855000,38.750000,37.520000,21.059999,25.360001,12.35,27.870001,...,19.559999,37.099998,3.22,3.49,652.510010,18.250000,41.709999,33.549999,19.440001,34.560001
2021-03-02,24.000000,25.940001,25.433001,25.999901,38.389999,36.570000,21.070000,25.309999,11.99,27.520000,...,20.879999,36.200001,3.22,3.15,640.869995,17.900000,41.020000,33.419998,19.900000,33.689999
2021-03-03,23.049999,25.965000,25.427601,25.886999,39.000000,35.939999,21.730000,25.320000,13.34,28.320000,...,20.450001,36.150002,3.39,3.00,638.940002,17.660000,41.610001,32.056999,20.150000,35.380001


In [15]:
positions=pd.DataFrame(np.dot(numUnits.values, np.expand_dims(result.evec[:, 0], axis=1).T)*c_data_test[[i,  k]].values)

ValueError: operands could not be broadcast together with shapes (181,2) (69,2) 

In [77]:
lookback=np.round(halflife).astype(int) #  setting lookback to the halflife found above
numUnits =-(yport-yport.rolling(lookback).mean())/yport.rolling(lookback).std() 

# la prossima riga da errore
positions=pd.DataFrame(np.dot(numUnits.values, np.expand_dims(result.evec[:, 0], axis=1).T)*c_data_test[[i,  k]].values)
pnl=np.sum((positions.shift().values)*(c_data_test[[i,  k]].pct_change().values), axis=1) # daily P&L of the strategy
ret=pnl/np.sum(np.abs(positions.shift()), axis=1)
cumret = pd.DataFrame((np.cumprod(1+ret)-1))

res[i+'/'+k] = {'position': positions, 'yport':yport, 'halflike': halflife, 'pnl': pnl, 'ret': ret,'cumret': cumret}



ValueError: operands could not be broadcast together with shapes (181,2) (69,2) 

In [ ]:
best = []
for i in res:
    if res[i]['cumret'].values[-1] > 0.4:
        best.append([i,res[i]['cumret'].values[-1][0],np.nansum(res[i]['pnl'])])

best